In [2]:
from __future__ import print_function
import utils.load_mnist as mnist
import numpy as np
import duti.duti as duti
from sklearn.preprocessing import MinMaxScaler
from sklearn.decomposition import PCA
from sklearn.pipeline import Pipeline

Using TensorFlow backend.


In [11]:
np.random.seed(42)
train_size = 500
test_size = train_size
pca_size = 3
X, Y, test_X, test_Y = mnist.load_mnist_17(train_size = train_size, test_size = test_size)
pipeline = Pipeline([('pca', PCA(n_components=pca_size)), ('scaling', MinMaxScaler(feature_range=(-1, 1)))])
pca_X = pipeline.fit_transform(X)
pca_test_X = pipeline.transform(test_X)

In [12]:
Y_flipped = mnist.flip_mnist_17(pca_X, Y, factor = 0.3, pca_size=pca_size)

In [13]:
flipped_score = mnist.evaluate_mnist(pca_X, Y_flipped, pca_test_X, test_Y, pca_size=pca_size)
print('Training with flipped data set:')
print('\tTest loss:', flipped_score[0])
print('\tTest accuracy:', flipped_score[1])

Training with flipped data set:
	Test loss: 0.436106041988
	Test accuracy: 0.956666666667


In [14]:
score = mnist.evaluate_mnist(pca_X, Y, pca_test_X, test_Y, pca_size=pca_size)
print('Training with correct data set:')
print('\tTest loss:', score[0])
print('\tTest accuracy:', score[1])

Training with correct data set:
	Test loss: 0.0748220643649
	Test accuracy: 0.978


In [23]:
# Original DUTI Workflow
remaining = np.random.choice(X.shape[0], size=X.shape[0], replace=False)
checking, remaining = remaining[0], remaining[1:]
checked = []
trusted_X, trusted_Y, confidence = [], [], []
ratio, score, corrected = [0], [flipped_score[1]], [0]

In [24]:
from utils.experiment_mnist import *

In [25]:
def generate_set_trusted(img, acc, corr, conf):
    def set_trusted(b):
        global remaining, checking, checked
        global trusted_X, trusted_Y, confidence, ratio, score, corrected
        trusted_X.append(pca_X[checking])
        trusted_Y.append([1.] if b.description == '7' else [0.])
        confidence.append(float(conf.value) * 2)
        delta = duti.regression(pca_X, np.array([Y_flipped]).T, np.array(trusted_X),
                                np.array(trusted_Y), np.array(confidence), 1e-6, 0.7)
        delta = np.sign(np.array([
            0 if (d[0] < 0 and v < 0.5) or (d[0] > 0 and v >= 0.5) or abs(d[0]) < 0.05
            else d[0] for (d, v) in zip(delta, Y_flipped)]))
        #curr_score = mnist.evaluate_mnist(pca_X, Y_flipped + delta, pca_test_X, test_Y, pca_size=pca_size)
        checked.append(checking)
        ratio.append(len(checked))
        #score.append(curr_score[1])
        incorrect = np.nonzero(Y_flipped + delta != Y)[0]
        incorrect = incorrect[~np.in1d(incorrect, np.array(checked))]
        corrected.append(np.sum(Y_flipped != Y) - incorrect.shape[0])
        checking, remaining = remaining[0], remaining[1:]
        display_image(X[checking], img)
        #display_plot(ratio, score, acc, 'Test accuracy')
        display_plot(ratio, corrected, corr, '# fixed')
    display_image(X[checking], img)
    #display_plot(ratio, score, acc, 'Test accuracy')
    display_plot(ratio, corrected, corr, '# fixed')
    return set_trusted
    
img, acc, corr, conf, b1, b7 = build_UI()
set_trusted = generate_set_trusted(img, acc, corr, conf)
b1.on_click(set_trusted)
b7.on_click(set_trusted)

SEJveChjaGlsZHJlbj0oVkJveChjaGlsZHJlbj0oSW1hZ2UodmFsdWU9JycsIGhlaWdodD0nMjgyJywgd2lkdGg9JzI4MicpLCBJbnRTbGlkZXIodmFsdWU9MCwgZGVzY3JpcHRpb249dSdDb27igKY=


In [29]:
np.savez(
    'output/duti_results', 
    ratio=np.array(ratio),
    corrected=np.array(corrected),
    trusted_X=np.array(trusted_X),
    trusted_Y=np.array(trusted_Y),
    confidence=np.array(confidence),
    checked=np.array(checked)
)

In [27]:
np.array(corrected)

array([ 0,  1,  1,  4,  8, 16, 30, 46, 57, 65, 74, 82, 86, 88, 90, 91, 90,
       92, 97, 94, 92, 91, 94, 91, 95, 93, 93, 92, 92, 91, 87, 88, 89, 89,
       90, 94, 94, 92, 93, 93, 94, 92, 95, 95])